# F1 Safety Car Predictor

## Authors: Petar Stamenković, Aleksa Mitrovčan

### Data Set

In [14]:
import pandas as pd
import os

# Define relative path to the CSV file
relative_path = os.path.join("archive", "races.csv")

# Read the CSV file using the relative path
races = pd.read_csv(relative_path)[['year', 'name', 'date']]

races


,year,name,date
0,2009,Australian Grand Prix,2009-03-29
1,2009,Malaysian Grand Prix,2009-04-05
2,2009,Chinese Grand Prix,2009-04-19
3,2009,Bahrain Grand Prix,2009-04-26
4,2009,Spanish Grand Prix,2009-05-10
...,...,...,...
1120,2024,Mexico City Grand Prix,2024-10-27
1121,2024,São Paulo Grand Prix,2024-11-03
1122,2024,Las Vegas Grand Prix,2024-11-23
1123,2024,Qatar Grand Prix,2024-12-01


### Data Preprocessing
In this step we saved all the races from 2014 to 2024 into a csv file, and added additional columns for precise data training. Than we added necessary data to the table.

In [17]:
# Data Preprocessing

races_filtered = races[races['year'] >= 2014]
races_filtered['weather'] = ''
races_filtered['num_dnfs'] = ''
races_filtered['circuit_type'] = ''
races_filtered['phy_diff'] = ''
races_filtered['safety_car']=''

# Save the filtered DataFrame to a new CSV file
#races_filtered.to_csv(''/home/alex64a/F1-Lap-Time-Predictor/archive/races_filtered.csv'', index=False)

print("File saved successfully!")

File saved successfully!


/tmp/ipykernel_28681/2376998003.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  races_filtered['weather'] = ''
/tmp/ipykernel_28681/2376998003.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  races_filtered['num_dnfs'] = ''
/tmp/ipykernel_28681/2376998003.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/